In [2]:
### load package
import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np
import gurobipy as gp
import time
model_sanitized_path = 'C:/Users/52427/Documents/Cases/Case11_Baixiang/Baixiang_supply_chain_optimization/model_sanitized/'
from datetime import date

### Load input tables

In [43]:
file_name = model_sanitized_path + 'model_input_20220616_EN_v3.xlsx'
xls = pd.ExcelFile(file_name)

customers_df = pd.read_excel(xls, 'Input_CustomerList')

sites_df = pd.read_excel(xls, 'Input_SiteList')

products_df = pd.read_excel(xls, 'Input_ProductList')

demand_df = pd.read_excel(xls, 'Input_CustomerDemand')

trans_policies_df = pd.read_excel(xls, 'Input_TransportationPolicy')

prod_policies_df = pd.read_excel(xls, 'Input_CapacityByLineByProduct')

prod_line_policies_df = pd.read_excel(xls, 'Input_CapacityByLine')

var_prod_cost_df = pd.read_excel(xls, 'Input_VariableProdCost')

var_handling_cost_df = pd.read_excel(xls, 'Input_VariableHandlingCost')

site_fixed_cost_df = pd.read_excel(xls, 'Input_SiteFixedCost')

inv_df = pd.read_excel(xls, 'Input_InventoryPolicy')

### load baseline data tables
hist_outbound_trans_df = pd.read_excel(xls, sheet_name='Input_HistCustomerFlows')

hist_inbound_trans_df = pd.read_excel(xls, sheet_name='Input_HistIntersiteFlows')

hist_prod_df = pd.read_excel(xls, sheet_name='Input_HistProduction')

xls.close()

In [44]:
### print data info
print('# customers:', customers_df.shape[0])
print('# sites:', sites_df.shape[0])
print('# products:', products_df.shape[0])
print('# customer-product combinations in demand table:', demand_df.shape[0])
print('# customers in demand table:', len(demand_df['CustomerName'].unique()))
print('# products in demand table:', len(demand_df['ProductName'].unique()))
print('# all lanes:', trans_policies_df.shape[0])
print('# origins in transportation table:', trans_policies_df[['Origin']].drop_duplicates().shape[0])
print('# destinations in transportation table:', trans_policies_df[['Destination']].drop_duplicates().shape[0])
print('# line-product combinations:',prod_policies_df.shape[0])
print('# lines:',prod_line_policies_df.shape[0])
print('# sites in factory fixed cost table:', site_fixed_cost_df.shape[0])
print('# products in factory variable production cost table:', len(var_prod_cost_df[['ProductName']].drop_duplicates()))

### print baseline data info
print('# dcs to customers:', hist_outbound_trans_df.shape[0])
print('# factories to dcs:', hist_inbound_trans_df.shape[0])
print('# production combs:', hist_prod_df.shape[0])

# customers: 293
# sites: 18
# products: 61
# customer-product combinations in demand table: 34115
# customers in demand table: 293
# products in demand table: 61
# all lanes: 2718
# origins in transportation table: 18
# destinations in transportation table: 302
# line-product combinations: 187
# lines: 59
# sites in factory fixed cost table: 18
# products in factory variable production cost table: 61
# dcs to customers: 34932
# factories to dcs: 506
# production combs: 1642


### Preprocess

In [45]:
### create lists of basic model elements
customers = [x for x in customers_df['CustomerName']]

dcs = [x for x in sites_df['SiteName'][sites_df['SiteName'].str.contains('DC')]]

factories = [x for x in sites_df['SiteName'][sites_df['SiteName'].str.contains('FAC')]]

products = [x for x in products_df['ProductName']]

months = range(1, 13)

In [46]:
### customer demand
cust_demand = demand_df.groupby(['CustomerName', 'ProductName', 'Period'])['CustomerDemand'].sum().to_dict()

### production capacity
line_product_rate = prod_policies_df.groupby(['FactoryName', 'ProductLine', 'ProductName'])['MachineHoursPerUnit'].sum().to_dict()

line_capacity_cap = prod_line_policies_df.groupby(['FactoryName', 'ProductLine'])['MachineHourCapacity'].sum().to_dict()

line_product_dict = prod_policies_df.groupby(['FactoryName','ProductLine'])['ProductName'].apply(list).to_dict()

### factory-attached warehouse storage capacity
site_storage_cap = sites_df.groupby(['SiteName'])['StorageCapacity'].max().to_dict()

### factory-attached warehouse handling capacity
site_handling_cap = sites_df.groupby(['SiteName'])['HandlingCapacity'].max().to_dict()

### factory variable production cost
var_prod_cost = var_prod_cost_df.groupby(['FactoryName', 'ProductName'])['VariableCost'].sum().to_dict()

### dc variable handling cost 
var_handling_cost = var_handling_cost_df.groupby(['SiteName'])['VariableCost'].sum().to_dict()

### factory fixed cost
site_fixed_cost = site_fixed_cost_df.groupby(['SiteName'])['FixedOperatingCost'].sum().to_dict()

### transportation cost
transp_cost = trans_policies_df.groupby(['Origin', 'Destination'])['Price'].max().to_dict()

### transportation distance
transp_dist = trans_policies_df.groupby(['Origin', 'Destination'])['Distance'].max().to_dict()

### unit product cubic meters
product_cubic = products_df.groupby(['ProductName'])['Volume'].max().to_dict()

### inventory turns
inv_turns = inv_df.groupby(['FactoryName'])['InventoryTurns'].max().to_dict()

KeyError: 'FactoryName'

In [ ]:
### historical factory-to-customer flows
hist_fc_flows = hist_outbound_trans_df.groupby(
    ['Origin', 'Destination', 'ProductName', 'Period'])['HistoricalShipment'].sum().to_dict()

### historical inter-factory flows
hist_ff_flows = hist_inbound_trans_df.groupby(
    ['Origin', 'Destination', 'ProductName', 'Period'])['HistoricalShipment'].sum().to_dict()

### historical production
hist_prod_flows = hist_prod_df.groupby(
    ['FactoryName', 'ProductLine', 'ProductName', 'Period'])['HistoricalProduction'].sum().to_dict()

In [ ]:
### generate index sets
ss_lanes = gp.tuplelist([(o, d, p, m) for o in factories for d in dcs for p in products for m in months]) 

sc_lanes = gp.tuplelist([(o, d, p, m) for o in dcs for (d, p, m) in cust_demand]) 

prod_policies_month_df = pd.merge(prod_policies_df[['FactoryName', 'ProductLine', 'ProductName']].assign(temp=1), 
                                  pd.DataFrame(months, columns =['Period']).assign(temp=1), on='temp').drop('temp', axis=1)
prod_policies = gp.tuplelist(zip(prod_policies_month_df['FactoryName'], prod_policies_month_df['ProductLine'], 
                                 prod_policies_month_df['ProductName'], prod_policies_month_df['Period']))
factory_lines = list(zip(prod_policies_month_df['FactoryName'], prod_policies_month_df['ProductLine']))

inv_basis = gp.tuplelist([(dc, p, m) for dc in dcs for p in products for m in list(months) + [max(months)+1]])

### per unit penalty cost of using dummy lanes
dmd_pen = np.ceil(max(var_prod_cost.values())*10 + max(var_handling_cost.values())*10 + max(transp_cost.values())*10)

### Scenario setting

In [ ]:
# scenarios matrix
scenario_list = ['Baseline2021','Unconstrainted','CloseOneFactory','Unconstrainted+RemoveStorageCons','Unconstrainted+RemoveHandlingCons']
constr_setting = {'dc_handling_cap':[False, True, True, True, False], 
                  'dc_storage_cap':[False, True, True, False, True],
                  'perc_demand_satisfied':[False, False, False, False, False],
                  'hist_fc':[True, False, False, False, False], 
                  'hist_ff':[True, False, False, False, False], 
                  'hist_prod':[True, False, False, False, False], 
                  'hist_no_prod':[True, False, False, False, False], 
                  'fix_factories_open':[True, True, True, True, True],
                  'fix_dcs_open':[True, True, True, True, True],
                  'num_factories_open':[9, 9, 8, 9, 9], 
                  'num_dcs_open':[9, 9, 8, 9, 9], 
                  'initial_inv': [True, True, True, True, True]}
scenario_matrix = pd.DataFrame(constr_setting, index=scenario_list)
scenario_matrix

,dc_handling_cap,dc_storage_cap,perc_demand_satisfied,hist_fc,hist_ff,hist_prod,hist_no_prod,fix_factories_open,fix_dcs_open,num_factories_open,num_dcs_open,initial_inv
Baseline2021,False,False,False,True,True,True,True,True,True,9,9,True
Unconstrainted,True,True,False,False,False,False,False,True,True,9,9,True
CloseOneFactory,True,True,False,False,False,False,False,True,True,8,8,True
Unconstrainted+RemoveStorageCons,True,False,False,False,False,False,False,True,True,9,9,True
Unconstrainted+RemoveHandlingCons,False,True,False,False,False,False,False,True,True,9,9,True


### Network Optimization Model

In [ ]:
today = date.today()
today = str(today).replace('-','')

#scenario = 'Baseline2021'
#scenario = 'Unconstrainted'
#scenario = 'CloseOneFactory'
#scenario = 'Unconstrainted+RemoveStorageCons'
scenario = 'Unconstrainted+RemoveHandlingCons'

In [ ]:
time_start = time.time()

### create model
m = gp.Model('BXNetworkOpt')

### decision variables
ss_flow = m.addVars(ss_lanes, vtype=gp.GRB.CONTINUOUS, name='ff_flow') # factory to dc flows
sc_flow = m.addVars(sc_lanes, vtype=gp.GRB.CONTINUOUS, name='fc_flow') # dc to customer flows
prod_flow = m.addVars(prod_policies, vtype=gp.GRB.CONTINUOUS, name='prod_flow') # production volume
inv_level = m.addVars(inv_basis, vtype=gp.GRB.CONTINUOUS, name='inv_level') # beginning inventory level of each month
f_open = m.addVars(factories, vtype=gp.GRB.BINARY, name='f_open') # factory open or not
dc_open = m.addVars(dcs, vtype=gp.GRB.BINARY, name='dc_open') # factory open or not
demand_slack = m.addVars(cust_demand, vtype=gp.GRB.BINARY, name='demand_slack') # slack variable for unmatched demand

m.update()

### objective function
tot_var_prod_cost = gp.quicksum(prod_flow[(f, l, p, m)]*var_prod_cost[(f, p)] for (f, l, p, m) in prod_policies)
tot_var_handling_cost = gp.quicksum(sc_flow[(o, d, p, m)]*var_handling_cost[(o)] for (o, d, p, m) in sc_lanes) 
tot_site_fixed_cost = gp.quicksum(f_open[f]*site_fixed_cost[f] for f in factories) + gp.quicksum(dc_open[dc]*site_fixed_cost[dc] for dc in dcs)
tot_ss_transp_cost = gp.quicksum(ss_flow[(o, d, p, m)]*product_cubic[p]*transp_cost[(o, d)] for (o, d, p, m) in ss_lanes)
tot_sc_transp_cost = gp.quicksum(sc_flow[(o, d, p, m)]*product_cubic[p]*transp_cost[(o, d)] for (o, d, p, m) in sc_lanes)
tot_dmd_pen_cost = demand_slack.sum('*', '*', '*')*dmd_pen
tot_cost = tot_var_prod_cost + tot_var_handling_cost + tot_site_fixed_cost + tot_ss_transp_cost + tot_sc_transp_cost + tot_dmd_pen_cost

m.setObjective(tot_cost, gp.GRB.MINIMIZE)

### demand satisfaction
m.addConstrs(
    (sc_flow.sum('*', c, p, m) == cust_demand[(c, p, m)] + demand_slack[(c, p, m)] for (c, p, m) in cust_demand.keys()), 'customer_demand'
)


### at least 80% demand must be satisfied
if scenario_matrix.loc[scenario, 'perc_demand_satisfied']:
    m.addConstr(
        (sc_flow.sum('*', '*', '*', '*') >= 0.8 * sum(cust_demand.values())), 'perc_demand_satisfied'
    )

### flow balance
### factory flow balance
m.addConstrs(
    (ss_flow.sum(f, '*', p, m) == prod_flow.sum(f, '*', p, m) for f in factories for p in products for m in months), 'flow_balance'
)
### dc flow balance
m.addConstrs(
    (inv_level[(dc, p, m)] + ss_flow.sum('*', dc, p, m) == sc_flow.sum(dc, '*', p, m) + inv_level[(dc, p, m+1)]
     for dc in dcs for p in products for m in months), 'flow_balance'
)

### factory production capacity by line
m.addConstrs(
    (gp.quicksum([prod_flow[(f, l, p, m)] * line_product_rate[(f, l, p)] for p in line_product_dict[(f, l)]])<= line_capacity_cap[(f, l)]*f_open[f] for (f, l, p, m) in prod_policies), 'line_capacity_cap'
)

### dc handling capacity
if scenario_matrix.loc[scenario, 'dc_handling_cap']:
    m.addConstrs(
        (sc_flow.sum(dc, '*', '*', m) 
         <= site_handling_cap[dc]*dc_open[dc] for dc in dcs for m in months), 'dc_handling_cap'
    )

### dc storage capacity
if scenario_matrix.loc[scenario, 'dc_storage_cap']:
    m.addConstrs(
        (inv_level.sum(dc, '*', m) 
         <= site_storage_cap[dc]*dc_open[dc] for dc in dcs for m in months), 'dc_storage_cap'
    )

### number of factories open
if scenario_matrix.loc[scenario, 'fix_factories_open']:
    m.addConstr(
        (f_open.sum('*') == scenario_matrix.loc[scenario, 'num_factories_open']), 'num_factories_open'
    )

### number of dcs open
if scenario_matrix.loc[scenario, 'fix_dcs_open']:
    m.addConstr(
        (dc_open.sum('*') == scenario_matrix.loc[scenario, 'num_dcs_open']), 'num_dcs_open'
    )

### zero inital inventory
if scenario_matrix.loc[scenario, 'initial_inv']:
    m.addConstrs(
        (inv_level[dc, p, 1] == 0 for dc in dcs for p in products), 'intial_inv'
    )

##########################historical cons
### historical factory-to-customer flows
if scenario_matrix.loc[scenario, 'hist_fc']:
    m.addConstrs(
        (sc_flow[(o, d, p, m)] == hist_fc_flows[(o, d, p, m)] for (o, d, p, m) in hist_fc_flows.keys()), 'hist_fc'
    )

### historical inter-factory flows
if scenario_matrix.loc[scenario, 'hist_ff']:
    m.addConstrs(
        (ss_flow[(o, d, p, m)] == hist_ff_flows[(o, d, p, m)] for (o, d, p, m) in hist_ff_flows.keys()), 'hist_ff'
    )

### historical production
if scenario_matrix.loc[scenario, 'hist_prod']:
    m.addConstrs(
        (prod_flow[(f, l, p, m)] == hist_prod_flows[(f, l, p, m)] for (f, l, p, m) in hist_prod_flows.keys()), 'hist_prod'
    )

### historical no production
if scenario_matrix.loc[scenario, 'hist_no_prod']:
    m.addConstrs(
        (prod_flow[(f, l, p, m)] == 0 for (f, l, p, m) in [x for x in prod_policies if x not in hist_prod_flows.keys()]), 
        'hist_no_prod'
    )



time_build_end = time.time()

### solve the model
m.optimize()
time_solve_end = time.time()

print()
print('optimal' if m.status==gp.GRB.OPTIMAL else 'infeasible')
print('model build time:', round(time_build_end - time_start), 's')
print('model solve time:', round(time_solve_end - time_build_end), 's')
tot_oprt_cost = tot_var_prod_cost.getValue() + tot_var_handling_cost.getValue() + tot_site_fixed_cost.getValue() \
                + tot_sc_transp_cost.getValue() + tot_ss_transp_cost.getValue()
print('total operating cost:', tot_oprt_cost)

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "800758-SCORteamCOMMON"
Set parameter CSAppName to value "SCOR Team Common License"
Set parameter LicenseID
Waiting for cloud server to start (pool 800758-SCORteamCOMMON)...
Starting...
Starting...
Starting...
Starting...
Starting...
Compute Server job ID: cd41a018-447f-4327-ae8b-7e9749f4e577
Capacity available on '800758-SCORteamCOMMON' cloud pool - connecting...
Established HTTPS encrypted connection
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Gurobi Compute Server Worker version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 50194 rows, 409841 columns and 800420 nonzeros
Model fingerprint: 0x91b9eb4a
Variable types: 375708 continuous, 34133 integer (34133 binary)
Coefficient statistics:
  Matrix range     [3e-03, 2e+05]
  Objective range  [1e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range 

### Optimization Results

In [ ]:
sc_flow_res = []

for i in sc_lanes:
    if sc_flow[i].x > 0:
        var_output = {
            'Origin': i[0],
            'Destination': i[1],
            'ProductName': i[2],
            'Period': i[3],
            'Quantity': sc_flow[i].x
        }
        sc_flow_res.append(var_output)

sc_flow_res_df = pd.DataFrame.from_records(sc_flow_res)

len(sc_flow_res_df) == len(hist_outbound_trans_df)

False

In [ ]:
ss_flow_res = []

for i in ss_lanes:
    if ss_flow[i].x > 0:
        var_output = {
            'Origin': i[0],
            'Destination': i[1],
            'ProductName': i[2],
            'Period': i[3],
            'Quantity': ss_flow[i].x
        }
        ss_flow_res.append(var_output)

ss_flow_res_df = pd.DataFrame.from_records(ss_flow_res)

len(ss_flow_res_df) == len(hist_inbound_trans_df)

False

In [ ]:
prod_flow_res = []

for i in prod_policies:
    if prod_flow[i].x > 0:
        var_output = {
            'FactoryName': i[0],
            'ProductLine': i[1],
            'ProductName': i[2],
            'Period': i[3],
            'Quantity': prod_flow[i].x
        }
        prod_flow_res.append(var_output)
        
prod_flow_res_df = pd.DataFrame.from_records(prod_flow_res)

len(prod_flow_res_df) == len(hist_prod_df)

False

In [ ]:
inv_level_res = []

for i in inv_basis:
    if inv_level[i].x > 0:
        var_output = {
            'SiteName': i[0],
            'ProductName': i[1],
            'Period': i[2],
            'Quantity': inv_level[i].x
        }
        inv_level_res.append(var_output)
        
inv_level_res_df = pd.DataFrame.from_records(inv_level_res)

inv_level_res_df.head()

,SiteName,ProductName,Period,Quantity
0,DC_1,PROD_6,3,2.680000e+02
1,DC_1,PROD_11,3,9.100000e+02
2,DC_1,PROD_33,13,3.637979e-12
3,DC_1,PROD_34,3,5.059617e+04
4,DC_1,PROD_39,9,2.098834e+05


In [ ]:
demand_slack_res = [] 

for i in cust_demand:
    if demand_slack[i].x > 0:
        var_output = {
            'FactoryName': i[1],
            'ProductName': i[2],
            'Period': i[3],
            'Quantity': demand_slack[i].x
        }
        demand_slack_res.append(var_output)
        
demand_slack_res_df = pd.DataFrame.from_records(demand_slack_res)

if len(demand_slack_res_df)>1:
    print(demand_slack_res_df[['Quantity']].sum())
else:
    print('no unsatisfied demand')

no unsatisfied demand


In [ ]:
f_open_res = []

for i in factories:
    var_output = {
        'SiteName': i,
        'SiteStatus': f_open[i].x
    }
    f_open_res.append(var_output)
        
f_open_res_df = pd.DataFrame.from_records(f_open_res)

f_open_res_df

,SiteName,SiteStatus
0,FAC_1,1.0
1,FAC_2,1.0
2,FAC_3,1.0
3,FAC_4,1.0
4,FAC_5,1.0
5,FAC_6,1.0
6,FAC_7,1.0
7,FAC_8,1.0
8,FAC_9,1.0


In [ ]:
dc_open_res = []

for i in dcs:
    var_output = {
        'SiteName': i,
        'SiteStatus': dc_open[i].x
    }
    dc_open_res.append(var_output)
        
dc_open_res_df = pd.DataFrame.from_records(dc_open_res)

dc_open_res_df

,SiteName,SiteStatus
0,DC_1,1.0
1,DC_2,1.0
2,DC_3,1.0
3,DC_4,1.0
4,DC_5,1.0
5,DC_6,1.0
6,DC_7,1.0
7,DC_8,1.0
8,DC_9,1.0


### Prepare output tables

In [85]:
### add cost of flow and production
sc_flow_res_df_1 = sc_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination'])\
    .merge(products_df[['ProductName','Volume']], on = ['ProductName'])\
        .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['Price']})\
            .merge(var_handling_cost_df.rename({'SiteName':'Origin'},axis=1), on = ['Origin'])\
                .assign(**{"HandlingCost": lambda x: x['Quantity']*x['VariableCost']})


ss_flow_res_df_1 = ss_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination'])\
    .merge(products_df[['ProductName','Volume']], on = ['ProductName'])\
        .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['Price']})

prod_flow_res_df_1 = prod_flow_res_df.merge(var_prod_cost_df, on = ['FactoryName','ProductName'])\
    .assign(**{"ProductionCost": lambda x: x['Quantity']*x['VariableCost']})\
        .merge(prod_policies_df, on = ['FactoryName','ProductLine','ProductName'])\
            .assign(**{'MachineHours': lambda x: x['Quantity'] * x['MachineHoursPerUnit']})

site_open_res_df_1 = site_fixed_cost_df.merge(pd.concat([f_open_res_df,dc_open_res_df]), on = 'SiteName', how= 'left')\
    .assign(**{'SiteFixedOperatingCost': lambda x: x['SiteStatus']*x['FixedOperatingCost']})

### period cost summary 
period_cost_summary = sc_flow_res_df_1.groupby(['Period'])[['TransportationCost','HandlingCost','Quantity']].sum().reset_index()\
    .rename({'TransportationCost':'TransportationCostCustomerFlows','HandlingCost':'DCHandlingCost','Quantity':'QuantityCustomerFlows'},axis=1)\
        .merge(ss_flow_res_df_1.groupby(['Period'])[['TransportationCost','Quantity']].sum().reset_index()\
            .rename({'TransportationCost':'TransportationCostIntersiteFlows','Quantity':'QuantityIntersiteFlows'},axis=1), on = ['Period'])\
                .assign(**{'TransportationCost': lambda x: (x['TransportationCostCustomerFlows']+x['TransportationCostIntersiteFlows'])})\
                    .merge(prod_flow_res_df_1.groupby(['Period'])[['ProductionCost','Quantity','MachineHours']].sum().reset_index()\
                        .rename({'ProductionCost':'FactoryProductionCost', 'Quantity':'FactoryProductionQuantity'},axis=1), on = ['Period'])\
                            .assign(SiteFixedOperatingCost = site_open_res_df_1[['FixedOperatingCost']].sum()[0]/12, 
                                    NumberOfSitesOpen = site_open_res_df_1[['SiteStatus']].sum()[0])\
                                        .assign(TotalCost = lambda x: x['TransportationCost']+x['DCHandlingCost']+x['FactoryProductionCost']+x['SiteFixedOperatingCost'])
                                            # .assign(ProductionCapacityUtilization = lambda x: x['MachineHours']/prod_line_policies_df['MachineHourCapacity'].sum(),
                                            #         HandlingCapacityUtilization = lambda x: x['QuantityCustomerFlows']/sites_df['HandlingCapacity'].sum(),
                                            #         ### use inventory turns to calculate monthly storage utilization, is it ok?
                                            #         StorageCapacityUtilization = lambda x: x['QuantityCustomerFlows']/(inv_df['InventoryTurns'].mean()/12*sites_df['StorageCapacity'].sum()))
  
### site cost summary
##### factory cost summary
factory_cost_summary = ss_flow_res_df_1.groupby(['Origin']).apply(lambda x : pd.Series({
                                                                                        'TransportationCost': x['TransportationCost'].sum(),
                                                                                        'Quantity': x['Quantity'].sum(),
                                                                                        'MaxDistance':x['Distance'].max(),
                                                                                        'WeightedAverageDistance':(x['Quantity']*x['Volume']*x['Distance']).sum() / (x['Quantity']*x['Volume']).sum(),
                                                                                        })).reset_index()\
            .rename({'Origin':'SiteName', 'TransportationCost':'OutboundTransportationCost', 'Quantity':'ThoughputLevel'},axis=1)\
                .merge(prod_flow_res_df_1.groupby(['FactoryName'])[['ProductionCost','MachineHours']].sum().reset_index()\
                    .merge(prod_line_policies_df.groupby(['FactoryName']).agg({'MachineHourCapacity':'sum'}).reset_index(), on = ['FactoryName'])\
                        .assign(MachineHourCapacity = lambda x: x['MachineHourCapacity']*12,
                                ProductionCapacityUtilization = lambda x: x['MachineHours']/x['MachineHourCapacity'])\
                            .rename({'FactoryName':'SiteName'},axis=1), on = 'SiteName')
factory_cost_summary.insert(1, 'SiteType', 'Factory')
##### dc cost summary
dc_cost_summary = sc_flow_res_df_1.groupby(['Origin']).apply(lambda x : pd.Series({'TransportationCost': x['TransportationCost'].sum(),
                                                                                    'HandlingCost': x['HandlingCost'].sum(),
                                                                                    'Quantity': x['Quantity'].sum(),
                                                                                    'MaxDistance':x['Distance'].max(),
                                                                                    'WeightedAverageDistance':(x['Quantity']*x['Volume']*x['Distance']).sum() / (x['Quantity']*x['Volume']).sum(),
                                                                                    })).reset_index()\
     .rename({'Origin':'SiteName', 'TransportationCost':'OutboundTransportationCost', 'Quantity':'ThoughputLevel'},axis=1)\
        .merge(sites_df[['SiteName','StorageCapacity','HandlingCapacity']], on = ['SiteName'])\
            .assign(HandlingCapacity = lambda x: x['HandlingCapacity']*12)\
                .merge(inv_df, on = ['SiteName'])\
                     .assign(HandlingCapacityUtilization = lambda x: x['ThoughputLevel']/x['HandlingCapacity'],
                             StorageCapacityUtilization = lambda x: x['ThoughputLevel']/(x['InventoryTurns']*x['StorageCapacity']))\
                                .drop(['InventoryTurns'],axis=1)
dc_cost_summary.insert(1, 'SiteType', 'Distribution Center')
site_cost_summary = pd.concat([factory_cost_summary, dc_cost_summary])

In [86]:
# #########################################(discard)
# ### dc to customer transportation cost
# sc_trans_cost_sum_df = sc_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination'])\
#     .merge(products_df, on = ['ProductName'])\
#         .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['Price']})\
#             .groupby(['Origin','ProductName','Period'])\
#                 .agg({'TransportationCost':'sum','Quantity':'sum'}).reset_index()\
#                     .rename({'Origin':'FactoryName'},axis=1)

# ### factory to dc transportation cost
# if ss_flow_res_df.shape[0] != 0:
#     ss_trans_cost_sum_df = ss_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination'])\
#         .merge(products_df, on = ['ProductName'])\
#             .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['Price']})\
#                  .groupby(['Destination','ProductName','Period'])\
#                     .agg({'TransportationCost':'sum','Quantity':'sum'}).reset_index()\
#                         .rename({'Destination':'FactoryName'},axis=1)
# else:
#     ff_trans_cost_sum_df = pd.DataFrame(columns=['FactoryName','Period','TransportationCost','Quantity'])

# ### factory variable cost
# factory_var_cost_sum_df = prod_flow_res_df.merge(factory_var_cost_df, on = ['FactoryName','ProductName'])\
#     .assign(**{"FactoryVariableCost": lambda x: x['Quantity']*x['VariableCost']})\
#         .merge(prod_policies_df, on = ['FactoryName','ProductLine','ProductName'])\
#             .assign(**{'MachineHours': lambda x: x['Quantity'] * x['MachineHoursPerUnit']})\
#                 .groupby(['FactoryName','ProductName','Period'])\
#                     .agg({'FactoryVariableCost':'sum','Quantity':'sum','MachineHours':'sum'}).reset_index()\
#                         .rename({'Quantity':'ProductionQuantity'},axis=1)

# ### factory fixed cost
# factory_fixed_cost_sum_df = pd.merge(f_open_res_df, factory_fixed_cost_df, on = 'FactoryName').assign(**{'FactoryFixedOperatingCost': lambda x: x['FactoryStatus']*x['FixedOperatingCost']})

# ### factory level cost summary
# factory_cost_sum_df = fc_trans_cost_sum_df.groupby(['FactoryName'])[['TransportationCostFactoryToCustomer', 'QuantityFactoryToCustomer']].sum().reset_index()\
#     .merge(ff_trans_cost_sum_df.groupby(['FactoryName'])[['TransportationCostAmongFactoryInflow','QuantityAmongFactoryInflow']].sum().reset_index(), on = ['FactoryName'], how = 'left').fillna(0)\
#         .merge(ff_flow_res_df.groupby(['Origin']).agg({'Quantity':'sum'}).reset_index().rename({'Origin':'FactoryName', 'Quantity':'QuantityAmongFactoryOutflow'},axis=1), on = ['FactoryName'], how = 'left').fillna(0)\
#             .merge(factory_var_cost_sum_df.groupby(['FactoryName'])[['FactoryVariableCost','ProductionQuantity','MachineHours']].sum().reset_index(), on = ['FactoryName'])\
#                 .merge(factory_fixed_cost_sum_df[['FactoryName','FactoryFixedOperatingCost']], on = ['FactoryName'])\
#                     .merge(factories_df[['FactoryName','StorageCapacity','HandlingCapacity']].assign(HandlingCapacity = lambda x: x['HandlingCapacity']*12).rename({'HandlingCapacity':'AnnualHandlingCapacity'},axis=1), on = ['FactoryName'])\
#                         .merge(inv_df[['FactoryName','InventoryTurns']].rename({'InventoryTurns':'AnnualInventoryTurns'},axis=1), on = ['FactoryName'])\
#                             .merge(prod_line_policies_df.groupby(['FactoryName']).agg(MachineHourCapacity = ('MachineHourCapacity', lambda x: x.sum() * 12)).reset_index().rename({'MachineHourCapacity':'AnnualMachineHourCapacity'},axis=1), on = ['FactoryName'])\
#                                 .assign(**{'HandlingCapacityUtilization': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])/x['AnnualHandlingCapacity']})\
#                                     .assign(**{'StorageCapacityUtilization': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])/(x['AnnualInventoryTurns']*x['StorageCapacity'])})\
#                                         .assign(**{'ProductionCapacityUtilization': lambda x: x['MachineHours']/x['AnnualMachineHourCapacity']})\
#                                             .assign(**{'TransportationQuantityOutflow': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])})\
#                                                 .assign(**{'TransportationCost': lambda x: (x['TransportationCostFactoryToCustomer']+x['TransportationCostAmongFactoryInflow'])})

# ### factory + month level cost summary
# factory_cost_monthly_sum_df = fc_trans_cost_sum_df.groupby(['FactoryName','Period'])[['TransportationCostFactoryToCustomer', 'QuantityFactoryToCustomer']].sum().reset_index()\
#     .merge(ff_trans_cost_sum_df.groupby(['FactoryName','Period'])[['TransportationCostAmongFactoryInflow','QuantityAmongFactoryInflow']].sum().reset_index(), on = ['FactoryName','Period'], how = 'outer')\
#         .merge(ff_flow_res_df.groupby(['Origin','Period']).agg({'Quantity':'sum'}).reset_index().rename({'Origin':'FactoryName', 'Quantity':'QuantityAmongFactoryOutflow'},axis=1), on = ['FactoryName','Period'], how = 'left').fillna(0)\
#             .merge(factory_var_cost_sum_df.groupby(['FactoryName','Period'])[['FactoryVariableCost','ProductionQuantity','MachineHours']].sum().reset_index(), on = ['FactoryName','Period'], how = 'outer')\
#                 .merge(factory_fixed_cost_sum_df[['FactoryName','FactoryFixedOperatingCost']].assign(**{'FactoryFixedOperatingCost':lambda x: x['FactoryFixedOperatingCost']/12}), on = ['FactoryName'], how = 'outer').fillna(0)\
#                     .merge(factories_df[['FactoryName','StorageCapacity','HandlingCapacity']].rename({'HandlingCapacity':'MonthlyHandlingCapacity'},axis=1), on = ['FactoryName'])\
#                         .merge(inv_df[['FactoryName','InventoryTurns']].assign(**{'MonthlyInventoryTurns': lambda x: x['InventoryTurns']/12}).drop(['InventoryTurns'],axis=1), on = ['FactoryName'])\
#                             .merge(prod_line_policies_df.groupby(['FactoryName']).agg({'MachineHourCapacity':'sum'}).reset_index().rename({'MachineHourCapacity':'MonthlyMachineHourCapacity'},axis=1), on = ['FactoryName'])\
#                                 .assign(**{'HandlingCapacityUtilization': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])/x['MonthlyHandlingCapacity']})\
#                                     .assign(**{'StorageCapacityUtilization': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])/(x['MonthlyInventoryTurns']*x['StorageCapacity'])})\
#                                         .assign(**{'ProductionCapacityUtilization': lambda x: x['MachineHours']/x['MonthlyMachineHourCapacity']})\
#                                             .assign(**{'TransportationQuantityOutflow': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])})\
#                                                     .assign(**{'TransportationCost': lambda x: (x['TransportationCostFactoryToCustomer']+x['TransportationCostAmongFactoryInflow'])})
                                                        
# factory_cost_monthly_sum_df['EndOfMonthInventory'] = factory_cost_monthly_sum_df.assign(**{'EndOfMonthInventory': lambda x: (x['ProductionQuantity']+x['QuantityAmongFactoryInflow']-x['QuantityAmongFactoryOutflow'])})\
#     .sort_values(['FactoryName','Period']).groupby(['FactoryName'])['EndOfMonthInventory'].cumsum()
# factory_cost_monthly_sum_df['StartOfMonthInventory'] = factory_cost_monthly_sum_df['EndOfMonthInventory'].shift(1).fillna(0)


# ### service level info
# service_level_df = pd.merge(sc_flow_res_df_1.groupby(['Origin','Period'])\
#             .apply(lambda x : pd.Series({'MaxDistanceFactoryToCustomer':x['Distance'].max(),
#             'WeightedAverageDistanceCustomerFlows':(x['Quantity']*x['Volume']*x['Distance']).sum() / (x['Quantity']*x['Volume']).sum(),
#             })).reset_index().rename({'Origin':'FactoryName'}, axis=1),
#         ss_flow_res_df_1.groupby(['Destination','Period'])\
#             .apply(lambda x : pd.Series({
#             'WeightedAverageDistanceAmongFactoryInflow':(x['Quantity']*x['Volume']*x['Distance']).sum() / (x['Quantity']*x['Volume']).sum(),
#             })).reset_index().rename({'Destination':'FactoryName'}, axis=1),
#         on = ['FactoryName','Period'],
#         how = 'outer')

### Save to Excel

In [87]:
### save model output template
writer = pd.ExcelWriter(model_sanitized_path + 'model_output_'+ today +'_'+scenario+'.xlsx', engine='xlsxwriter')

### output data tables
pd.DataFrame({'Scenario': [scenario],
                'TotalCost':[tot_var_prod_cost.getValue()+tot_var_handling_cost.getValue()+tot_site_fixed_cost.getValue()+tot_sc_transp_cost.getValue()+tot_ss_transp_cost.getValue()],
                'FactoryVariableProductionCost':[tot_var_prod_cost.getValue()],
                'DCVariableHandlingCost':[tot_var_handling_cost.getValue()],
                'SiteFixedOperatingCost':[tot_site_fixed_cost.getValue()],
                'TransportationCost':[tot_sc_transp_cost.getValue()+tot_ss_transp_cost.getValue()],
                'TransportationCostCustomerFlows':[tot_sc_transp_cost.getValue()],
                'TransportationCostIntersiteFlows':[tot_ss_transp_cost.getValue()],
                'DemandSatisfiedPerc': [sc_flow_res_df_1['Quantity'].sum() / demand_df['CustomerDemand'].sum()],
                'TotalDemand':[demand_df['CustomerDemand'].sum()],
                'QuantityCustomerFlows':[sc_flow_res_df_1['Quantity'].sum()],
                'QuantityIntersiteFlows':[ss_flow_res_df_1['Quantity'].sum()],
                'MaxDistanceCustomerFlows':[sc_flow_res_df_1['Distance'].max()],
                'WeightedAverageDistanceCustomerFlows':[(sc_flow_res_df_1['Quantity']*sc_flow_res_df_1['Volume']*sc_flow_res_df_1['Distance']).sum() / (sc_flow_res_df_1['Quantity']*sc_flow_res_df_1['Volume']).sum()],
                'WeightedAverageDistanceIntersiteFlows':[(ss_flow_res_df_1['Quantity']*ss_flow_res_df_1['Volume']*ss_flow_res_df_1['Distance']).sum() / (ss_flow_res_df_1['Quantity']*ss_flow_res_df_1['Volume']).sum()]
                })\
        .to_excel(writer, sheet_name='Output_CostSummary', index=False)
period_cost_summary.insert(0, 'Scenario', scenario)
period_cost_summary.to_excel(writer, sheet_name='Output_CostSummaryByPeriod', index=False)
site_cost_summary.insert(0, 'Scenario', scenario)
site_cost_summary.to_excel(writer, sheet_name='Output_CostSummaryBySite', index=False)
        
sc_flow_res_df_1.assign(**{'Scenario': scenario})[['Scenario','Origin','Destination','ProductName','Period','Quantity','TransportationCost','HandlingCost']].to_excel(writer, sheet_name='Output_CustomerFlows', index=False)
ss_flow_res_df_1.assign(**{'Scenario': scenario})[['Scenario','Origin','Destination','ProductName','Period','Quantity','TransportationCost']].to_excel(writer, sheet_name='Output_IntersiteFlows', index=False)
prod_flow_res_df_1.assign(**{'Scenario': scenario})[['Scenario','FactoryName', 'ProductLine', 'ProductName', 'Period', 'Quantity', 'ProductionCost']].to_excel(writer, sheet_name='Output_ProductionFlows', index=False)

inv_level_res_df.assign(**{'Scenario': scenario})[['Scenario','SiteName','ProductName','Period','Quantity']].to_excel(writer, sheet_name='Output_InventoryLevel', index=False)

### need further cleaning format before send to client
### input data tables
products_df.to_excel(writer, sheet_name='Input_ProductList', index=False)
customers_df.to_excel(writer, sheet_name='Input_CustomerList', index=False)
sites_df.to_excel(writer, sheet_name='Input_FactoryList', index=False)
var_prod_cost_df.to_excel(writer, sheet_name='Input_FactoryVariableCost', index=False)
site_fixed_cost_df.to_excel(writer, sheet_name='Input_FactoryFixedCost', index=False)
trans_policies_df.to_excel(writer, sheet_name='Input_TransportationPolicy', index=False)
prod_policies_df.to_excel(writer, sheet_name='Input_CapacityByLineByProduct', index=False)
prod_line_policies_df.to_excel(writer, sheet_name='Input_CapacityByLine', index=False)
inv_df.to_excel(writer, sheet_name='Input_InventoryPolicy', index=False)
demand_df.to_excel(writer, sheet_name='Input_CustomerDemand', index=False)
hist_outbound_trans_df.to_excel(writer, sheet_name='Input_HistShipmentFactoryToCust', index=False)
hist_inbound_trans_df.to_excel(writer, sheet_name='Input_HistShipmentAmongFactory', index=False)
hist_prod_df.to_excel(writer, sheet_name='Input_HistProduction', index=False)

writer.save()
writer.close()

c:\Users\52427\.conda\envs\gurobienv\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [22]:
time.time()

1655295201.3283813